In [ ]:
!git clone https://github.com/JeffersonEspinalA/TF-ML

In [ ]:
!pip install chart_studio
!pip install numpy-stl
!pip install trimesh
!pip install scikit-image

In [19]:
import plotly.io as pio
pio.renderers.default = "colab"

In [ ]:
import numpy as np
from TF_ML.reconstruction.utils.plot import plot_vol
from TF_ML.reconstruction.utils.data_prep import get_fractured
from stl import mesh
from skimage.measure import marching_cubes

### Escoger el objeto y romper el objeto

In [ ]:
data = np.load('/content/TF_ML/ModelNet10/voxels.npy',allow_pickle=True).item()['test']['data']

In [ ]:
completo = data[168]
fracturado = get_fractured(completo)

In [20]:
print('Fracturado:')
plot_vol(fracturado)
print('Completo:')
plot_vol(completo)

Fracturado:


Completo:


### Exportar a archivo STL

In [ ]:
def convert_to_binary_voxel(input_voxel):
    result = np.zeros((32, 32, 32))
    for i in range(32):
        for j in range(32):
            for k in range(32):
                if(input_voxel[i][j][k] == 1.0):
                    result[i][j][k] = 1
                else:
                    result[i][j][k] = 0
    return result

In [ ]:
result_voxel_fracturado = convert_to_binary_voxel(fracturado)

vertices, faces, _, _ = marching_cubes(result_voxel_fracturado, level=0)
mesh_data = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
for i, face in enumerate(faces):
    for j in range(3):
        mesh_data.vectors[i][j] = vertices[face[j], :]

mesh_data.save('resultado_fracturado.stl')

In [ ]:
result_voxel_completo = convert_to_binary_voxel(completo)

vertices, faces, _, _ = marching_cubes(result_voxel_completo, level=0)
mesh_data = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
for i, face in enumerate(faces):
    for j in range(3):
        mesh_data.vectors[i][j] = vertices[face[j], :]

mesh_data.save('resultado_completo.stl')

In [ ]:
agujero = np.logical_xor(result_voxel_fracturado, result_voxel_completo)

vertices, faces, _, _ = marching_cubes(agujero, level=0)
mesh_data = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
for i, face in enumerate(faces):
    for j in range(3):
        mesh_data.vectors[i][j] = vertices[face[j], :]

mesh_data.save('resultado_agujero.stl')